In [1]:
import os
import sys
sys.path.append("/home/pervinco/mmsegmentation")

import cv2
import yaml
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import albumentations as A
import torch.nn.functional as F

from glob import glob
from tqdm import tqdm
from datetime import datetime
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

from mmseg.models import build_segmentor
from data_preprocess import load_img_mask, encode_mask, train_img_mask_transform, mosaic_augmentation, spatially_exclusive_pasting

In [2]:
def save_config_to_yaml(config, save_dir):
    with open(f"{save_dir}/params.yaml", 'w') as file:
        yaml.dump(config, file)

In [3]:
class BKAIDataset(Dataset):
    def __init__(self, config):
        self.data_dir = config["data_dir"]
        self.img_size = config["img_size"]
        self.spatial_alpha = config["spatial_alpha"]

        self.image_dir = f"{self.data_dir}/train"
        self.mask_dir = f"{self.data_dir}/train_gt"

        self.image_files = sorted(glob(f"{self.image_dir}/*.jpeg"))
        self.mask_files = sorted(glob(f"{self.mask_dir}/*.jpeg"))
        
        self.total_files = list(zip(self.image_files, self.mask_files))

        self.train_transform = A.Compose([A.HorizontalFlip(p=0.5),
                                          A.VerticalFlip(p=0.5),
                                          A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
                                          A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
                                          A.OneOf([A.Blur(), A.GaussianBlur(), A.GlassBlur(), A.MotionBlur(), A.GaussNoise(), A.Sharpen(), A.MedianBlur(), A.MultiplicativeNoise()]),
                                          A.Cutout(p=0.2, max_h_size=35, max_w_size=35, fill_value=255),
                                          A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.15, brightness_coeff=1.5, p=0.09),
                                          A.RandomShadow(p=0.1),
                                          A.ShiftScaleRotate(p=0.45, border_mode=cv2.BORDER_CONSTANT, shift_limit=0.15, scale_limit=0.15),
                                          A.RandomCrop(self.img_size, self.img_size)])
        
        self.batch_transform = A.Compose([A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
                                          ToTensorV2()])

    def __len__(self):
        return len(self.total_files)
    

    def __getitem__(self, index):
        prob = random.random()

        if prob <= 0.3:
            image_path, mask_path = self.total_files[index]
            image, mask = load_img_mask(image_path, mask_path, size=self.img_size)

            augment_image, augment_mask = train_img_mask_transform(self.train_transform, image, mask)

        elif 0.3 < prob <= 0.6:
            piecies = []
            while len(piecies) < 4:
                i = random.randint(0, len(self.total_files)-1)
                image_path, mask_path = self.total_files[i]
                image, mask = load_img_mask(image_path, mask_path, size=self.img_size)
                piece_image, piece_mask = train_img_mask_transform(self.train_transform, image, mask)
                piecies.append([piece_image, piece_mask])

            augment_image, augment_mask = mosaic_augmentation(piecies, size=self.img_size)


        elif 0.6 < prob <= 1:
            image_path, mask_path = self.total_files[index]
            image, mask = load_img_mask(image_path, mask_path, size=self.img_size)
            augment_image, augment_mask = spatially_exclusive_pasting(image, mask, alpha=random.uniform(self.spatial_alpha, self.spatial_alpha + 0.2))

        encoded_mask = encode_mask(augment_mask)
        batch_image, batch_mask = train_img_mask_transform(self.batch_transform, augment_image, encoded_mask)

        return batch_image, batch_mask

In [4]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

## Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_workers = min([os.cpu_count(), config["batch_size"] if config["batch_size"] > 1 else 0, 8])

## Dataset
train_dataset = BKAIDataset(config)
train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=num_workers)

/home/pervinco/anaconda3/envs/openmmlab/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [5]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [6]:
#metrics
#tham khao: https://github.com/hszhao/semseg/blob/master/util/util.py
def intersectionAndUnionGPU(output, target, K, ignore_index=255):
    # 'K' classes, output and target sizes are N or N * L or N * H * W, each value in range 0 to K - 1.
    assert (output.dim() in [1, 2, 3])
    assert output.shape == target.shape
    output = output.view(-1)
    target = target.view(-1)
    output[target == ignore_index] = ignore_index
    intersection = output[output == target]
    area_intersection = torch.histc(intersection, bins=K, min=0, max=K-1)
    area_output = torch.histc(output, bins=K, min=0, max=K-1)
    area_target = torch.histc(target, bins=K, min=0, max=K-1)
    area_union = area_output + area_target - area_intersection
    return area_intersection, area_union, area_target 

In [7]:
class FocalLoss_Ori(nn.Module):
    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

    def forward(self, logit, target):
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()
        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss

In [8]:
#device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

norm_cfg = dict(type='BN', requires_grad=True)
# checkpoint = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/segformer/mit_b4_20220624-d588d980.pth'  # noqa
# model = dict(
#     type='EncoderDecoder',
#     pretrained=None,
#     backbone=dict(
#         type='MixVisionTransformer',
#         in_channels=3,
#         embed_dims=64,
#         num_stages=4,
#         num_layers=[3, 8, 27, 3],
#         num_heads=[1, 2, 5, 8],
#         patch_sizes=[7, 3, 3, 3],
#         sr_ratios=[8, 4, 2, 1],
#         out_indices=(0, 1, 2, 3),
#         mlp_ratio=4,
#         qkv_bias=True,
#         drop_rate=0.0,
#         attn_drop_rate=0.0,
#         drop_path_rate=0.,
#         init_cfg = dict(type="Pretrained", checkpoint=checkpoint)),
#     decode_head=dict(
#         type='SegformerHead',
#         in_channels=[64, 128, 320, 512],
#         in_index=[0, 1, 2, 3],
#         channels=256,
#         dropout_ratio=0.1,
#         num_classes=config["num_classes"],
#         norm_cfg=norm_cfg,
#         align_corners=False,
#         loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
#     train_cfg=dict())

backbone_norm_cfg = dict(type='LN', requires_grad=True)
# checkpoint_file = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/swin/swin_large_patch4_window12_384_22k_20220412-6580f57d.pth'  # noqa
# model_cfg = dict(
#     type='EncoderDecoder',
#     pretrained=None,
#     backbone=dict(
#         type='SwinTransformer',
#         pretrain_img_size=config["img_size"],
#         embed_dims=192,
#         patch_size=4,
#         mlp_ratio=4,
#         window_size=12,
#         depths=[2, 2, 18, 2],
#         num_heads=[6, 12, 24, 48],
#         strides=(4, 2, 2, 2),
#         out_indices=(0, 1, 2, 3),
#         qkv_bias=True,
#         qk_scale=None,
#         patch_norm=True,
#         drop_rate=0.,
#         attn_drop_rate=0.,
#         drop_path_rate=0.3,
#         use_abs_pos_embed=False,
#         act_cfg=dict(type='GELU'),
#         norm_cfg=backbone_norm_cfg,
#         init_cfg=dict(type='Pretrained', checkpoint=checkpoint_file)),
#     decode_head=dict(
#         type='UPerHead',
#         in_channels=[192, 384, 768, 1536],
#         in_index=[0, 1, 2, 3],
#         pool_scales=(1, 2, 3, 6),
#         channels=512,
#         dropout_ratio=0.1,
#         num_classes=config["num_classes"],
#         norm_cfg=norm_cfg,
#         align_corners=False,
#         loss_decode=dict(
#             type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
#     auxiliary_head=dict(
#         type='FCNHead',
#         in_channels=768,
#         in_index=2,
#         channels=256,
#         num_convs=1,
#         concat_input=False,
#         dropout_ratio=0.1,
#         num_classes=config["num_classes"],
#         norm_cfg=norm_cfg,
#         align_corners=False,
#         loss_decode=dict(
#             type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
#     # model training and testing settings
#     train_cfg=dict(),
#     test_cfg=dict(mode='whole'))

checkpoint_file = 'https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/swin/swin_large_patch4_window7_224_22k_20220412-aeecf2aa.pth'  # noqa
model_cfg = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=config["img_size"],
        embed_dims=192,
        patch_size=4,
        mlp_ratio=4,
        window_size=12,
        depths=[2, 2, 18, 2],
        num_heads=[6, 12, 24, 48],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.,
        attn_drop_rate=0.,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=backbone_norm_cfg,
        init_cfg=dict(type='Pretrained', checkpoint=checkpoint_file)),
    decode_head=dict(
        type='UPerHead',
        in_channels=[192, 384, 768, 1536],
        in_index=[0, 1, 2, 3],
        pool_scales=(1, 2, 3, 6),
        channels=512,
        dropout_ratio=0.1,
        num_classes=config["num_classes"],
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    auxiliary_head=dict(in_channels=768, num_classes=config["num_classes"]))

model = build_segmentor(model_cfg).to(device)
model.init_weights()

#loss
criterion = FocalLoss_Ori(num_class=3)

#optimizer
n_eps = config["epochs"]
init_lr = config["init_lr"]
optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader)*n_eps, eta_min=init_lr/100)
total_step = len(train_dataloader)

#meter
train_loss_meter = AverageMeter()
intersection_meter = AverageMeter()
union_meter = AverageMeter()
target_meter = AverageMeter()

/home/pervinco/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/pervinco/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmsegmentation/v0.5/pretrain/swin/swin_large_patch4_window12_384_22k_20220412-6580f57d.pth
10/10 15:52:48 - mmengine - INFO - 
backbone.patch_embed.projection.weight - torch.Size([192, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
10/10 15:52:48 - mmengine - INFO - 
backbone.patch_embed.projection.bias - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
10/10 15:52:48 - mmengine - INFO - 
backbone.patch_embed.norm.weight - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
10/10 15:52:48 - mmengine - INFO - 
backbone.patch_embed.norm.bias - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
10/10 15:52:48 - mmengine - INFO - 
backbone.stages.0.blocks.0.norm1.weight - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
10/10 15:52:48 - mmengine

In [9]:
## Save Config
save_dir = config["save_dir"]
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
save_path = f"{save_dir}/{current_time}"

if not os.path.isdir(save_path):
    print(save_path)
    os.makedirs(f"{save_path}/weights")
    config["save_dir"] = save_path

save_config_to_yaml(config, save_path)

./runs/2023-10-10_15-52-48


In [10]:
max_dice = 0.0 
for ep in range(1, 1+n_eps):
    train_loss_meter.reset()
    intersection_meter.reset()
    union_meter.reset()
    target_meter.reset()
    model.train()

    for batch_id, (x, y) in enumerate(tqdm(train_dataloader), start=1):
        if ep <= 1:
            optimizer.param_groups[0]["lr"] = (ep * batch_id) / (1.0 * total_step) * init_lr
        else:
            scheduler.step()
        optimizer.zero_grad()
        n = x.shape[0]
        x = x.to(device).float()
        y = y.to(device).long()
        # print(x.shape, y.shape)
        
        y_hat = model.forward_dummy(x) #(B, C, H, W)
        # print(y_hat.shape)

        loss = criterion(y_hat, y) #(B, C, H, W) >< (B, H, W)
        loss.backward()
        optimizer.step()

        #save metrics
        with torch.no_grad():
            train_loss_meter.update(loss.item())
            y_hat_mask = y_hat.argmax(dim=1).squeeze(1) # (B, C, H, W) -> (B, 1, H, W) -> (B, H, W)
            intersection, union, target = intersectionAndUnionGPU(y_hat_mask.float(), y.float(), 3)
            intersection_meter.update(intersection)
            union_meter.update(union)
            target_meter.update(target)

    #compute iou, dice
    with torch.no_grad():
        iou_class = intersection_meter.sum / (union_meter.sum + 1e-10) #vector 3D
        dice_class = (2 * intersection_meter.sum) / (intersection_meter.sum + union_meter.sum + 1e-10) #vector 3D

        mIoU = torch.mean(iou_class[1:]) #mean iou class 1 and class 2
        mDice = torch.mean(dice_class[1:]) #mean dice class 1 and class 2

    print(f"EP : {ep}, current_lr = {scheduler.get_last_lr()}, train_loss = {train_loss_meter.avg:.4f}, IoU = {mIoU:.4f}, dice = {mDice:.4f} \n")

    if mDice > max_dice:
        max_dice = mDice
        torch.save(model.state_dict(), f"{save_path}/weights/best_dice_ckpt_ep_{ep}.pth")

100%|██████████| 125/125 [00:52<00:00,  2.38it/s]


EP : 1, current_lr = [0.0001], train_loss = 0.2471, IoU = 0.1459, dice = 0.2317 



100%|██████████| 125/125 [00:51<00:00,  2.42it/s]


EP : 2, current_lr = [9.997557473810375e-05], train_loss = 0.0505, IoU = 0.3796, dice = 0.4892 



100%|██████████| 125/125 [00:52<00:00,  2.39it/s]


...

EP : 98, current_lr = [1.2196827521475498e-06], train_loss = 0.0038, IoU = 0.9067, dice = 0.9507 



100%|██████████| 125/125 [00:51<00:00,  2.41it/s]


EP : 99, current_lr = [1.0976769428005625e-06], train_loss = 0.0039, IoU = 0.9066, dice = 0.9505 



100%|██████████| 125/125 [00:51<00:00,  2.42it/s]

EP : 100, current_lr = [1.0244252618962874e-06], train_loss = 0.0039, IoU = 0.9065, dice = 0.9505 

